In [1]:
import requests
import re
import pandas as pd
import xlrd
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import time
import FinanceDataReader as fdr

In [2]:
crtfc_key = open('crtfc_key.txt',mode='r').readline()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'

# Corpcode

## Corpcode.xml 가져오기

In [13]:
url_corp = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key='+crtfc_key
# requests.get(url_corp).content.decode("UTF-8")
with urlopen(url_corp) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('Raw/Corpcode/corp_num')

## Corpcode.xml에서 기업코드 가져오기

In [151]:
tree = ET.parse('Raw/Corpcode/corp_num/CORPCODE.xml')
root = tree.getroot()

corp_df = pd.DataFrame(columns=['corp_code','corp_name','stock_code','modify_date'])
for company in root.iter('list'):
    stock_code = company.findtext('stock_code')
    stock_code = stock_code.strip()
    if stock_code:
        company_dict = {
            'corp_code':company.findtext('corp_code'),
            'corp_name':company.findtext('corp_name'),
            'stock_code':company.findtext('stock_code'),
            'modify_date':company.findtext('modify_date')
        }
        corp_df = corp_df.append(company_dict, ignore_index=True)
# corp_df

In [ ]:
# corp_df[corp_df['corp_name']=='BYC']
a = corp_df[corp_df['stock_code']=='168490'].iloc[0,0]
corp_df[corp_df['corp_code']==a]

# 필요 함수

In [137]:
rcp_no_list = rcp_df.iloc[0,2:][0:3].to_list()

In [138]:
rcp_no_list

[20210309000744, 20201116001248, 20200814001766]

In [141]:
test=list(map(making_dcm_eleid,rcp_no_list))

In [154]:
making_rcp_df(stock_code_reset('036670'))

,stock_code,description,2020.12,2020.09,2020.06,2020.03,2019.12
0,036670,rcp_no,20210312000686,20201116001598,20200814001492,20200515001672,20200330002806
1,036670,dcm_no,7871233,7549913,7445216,7341968,7196014
2,036670,eleid,14,13,13,13,13


In [149]:
def making_rcp_df(corp_code):
#     bgn_de = "19990101"
    bgn_de = "20200101"
    url_api = "https://opendart.fss.or.kr/api/list.xml?crtfc_key=" + crtfc_key + "&corp_code=" + corp_code + "&bgn_de=" + bgn_de +"&last_reprt_at=Y" + "&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003" + "&page_no=10&page_count=100"
    dart_page = requests.get(url_api).content.decode("UTF-8")
    rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', dart_page)
    period_list = re.findall(r'<report_nm>(.*?)</report_nm>', dart_page)
    year_list = list(map(year_reset, period_list))
    
#     dcm_no_list = list(map(making_dcm, rcp_no_list))
    dcm_eleid_no = list(map(making_dcm_eleid,rcp_no_list))
    dcm_no_list = pd.DataFrame(dcm_eleid_no)[0].to_list()
    eleid_list = pd.DataFrame(dcm_eleid_no)[1].to_list()
    
    stock_code = corp_code_reset(corp_code)
    
    rcp_no_list.insert(0,stock_code)
    rcp_no_list.insert(1,'rcp_no')
    year_list.insert(0,'stock_code')
    year_list.insert(1,'description')
    dcm_no_list.insert(0,stock_code)
    dcm_no_list.insert(1,'dcm_no')
    eleid_list.insert(0,stock_code)
    eleid_list.insert(1,'eleid')
    
#     rcp_df = pd.DataFrame(data=[rcp_no_list,dcm_no_list],columns=year_list)
    rcp_df = pd.DataFrame(data=[rcp_no_list,dcm_no_list,eleid_list],columns=year_list)
    rcp_df = rcp_df.loc[:, ~rcp_df.columns.duplicated(
        keep='first')]  #한 해에 여러개의 보고서가 나오면 최근거만 가져옴
    return rcp_df

def making_dcm (rcp_no):
    url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    resp = requests.get(url,headers={'user-agent':user_agent})
    webpage = resp.content.decode("UTF-8")
    dcm_no = re.findall(r"{}', '(.*?)',".format(rcp_no),webpage)[0]
#     print(dcm_no)
    time.sleep(0.5)
    return dcm_no

def making_dcm_eleid (rcp_no):
    url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    resp = requests.get(url,headers={'user-agent':user_agent})
    webpage = resp.content.decode("UTF-8")
    webpage_clean = ['\n','\t',' ','(',')',"'",'"']
    for clean in webpage_clean:
        webpage = webpage.replace(clean,'')
    try:
        webpage = re.findall(r"연결재무제표(.*?).dtd",webpage)[0]
    except:
        webpage = re.findall(r"재무제표(.*?).dtd",webpage)[0]
    webpage = re.findall(r"viewDoc(.*)",webpage)[0]
    dcm_eleid_no = webpage.split(',')[1:3]
    time.sleep(0.5)
    return dcm_eleid_no

def corp_code_reset(corp_code):  #에러가 발생할 경우 회사 코드 불러오기 실핸한다.
    stock_code = corp_df[corp_df['corp_code'] == corp_code].iloc[0, 2]
    return stock_code

def stock_code_reset(stock_code):
    corp_code = corp_df[corp_df['stock_code'] == stock_code].iloc[0, 0]
    return corp_code

def year_reset(title):
    try:
        year = [i for i in re.findall('\(([^)]+)', title) if i][0]
    except:
        year = '0'
    return year


In [93]:
from bs4 import BeautifulSoup

# rcp, dcm 가져오기

## stock_code 가져오기

In [6]:
# stock_df = fdr.StockListing('KOSPI')
stock_code_list = ['005930','005380','017670']

## rcp, dcm, eleid 만들기

In [40]:
rcp_df

,stock_code,description,2020.12,2020.09,2020.06,2020.03,2019.12,2019.09,2019.06,2019.03,...,2001.09,2001.06,2000.12,2001.03,2000.09,2000.06,1999.12,2000.03,1999.06,1998.12
0,005930,rcp_no,20210309000744,20201116001248,20200814001766,20200515001451,20200330003851,20191114001273,20190814002218,20190515001605,...,20011114000621,20010814001128,20010331000297,20010515000548,20001114000729,20000814000482,20000330000796,20000515000739,19990824000003,19990626000007
1,005930,dcm_no,7861003,7549051,7446167,7341446,7202455,6958001,6846651,6738798,...,263890,237424,178927,206571,113424,89871,41243,60524,22086,15402


In [7]:
# start = time.time()
rcp_df = pd.DataFrame()
indexerror_list=[]
valueerror_list=[]
for i in stock_code_list:
    try :
        rcp_df_temp = making_rcp_df(stock_code_reset(i))
        rcp_df = pd.concat([rcp_df,rcp_df_temp],axis=0)
    except IndexError:
        indexerror_list.append(i)
    except ValueError:
        valueerror_list.append(i)
    except Exception as e:
        print (i,e)
rcp_df.reset_index(drop=True,inplace=True)
# print(time.time()-start)
rcp_df.to_csv('Raw/rcp_dcm/rcp_df.csv', index=True, encoding='utf-8-sig')
# IndexError : single positional indexer is out-of-bounds : 우선주 등 재무제표 없는 경우
# list index out of range : 보고서 제목에 년도가 안써있는 경우 -> 0으로 쓰는것으로 수정
# ValueError : Length mismatch: Expected axis has 0 elements, new values have 1 elements : 리츠 등 재무제표 없는 경우

005380 ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


KeyboardInterrupt: 

## rcp, dcm 불러오기

In [128]:
def six_degree (stock_code):
    stock_code_return = '%06d'%(stock_code)
    return stock_code_return

In [129]:
rcp_df = pd.read_csv('Raw/rcp_dcm/rcp_df.csv')
# rcp_df = rcp_df.set_index(rcp_df.columns[0])
# rcp_df.index.name = None
rcp_df = rcp_df.drop(columns=rcp_df.columns[0])
rcp_df.reset_index(drop=True,inplace=True)
# column 지정
# rcp_df.columns = [c.columns.map(lambda x : x[:2]), c.iloc[0], c.iloc[1]]
# c = c.iloc[2:]
# 종목명 6자리로 변경
# for i in range(len(rcp_df.index)):
#     rcp_df.rename(index={rcp_df.index[i]:'%06d'%(rcp_df.index[i])}, inplace = True)
rcp_df['stock_code']=list(map(six_degree, rcp_df['stock_code']))

rcp_df

,stock_code,description,2020.12,2020.09,2020.06,2020.03,2019.12,2019.09,2019.06,2019.03,...,2001.09,2001.06,2000.12,2001.03,2000.09,2000.06,1999.12,2000.03,1999.06,1998.12
0,005930,rcp_no,20210309000744,20201116001248,20200814001766,20200515001451,20200330003851,20191114001273,20190814002218,20190515001605,...,20011114000621,20010814001128,20010331000297,20010515000548,20001114000729,20000814000482,20000330000796,20000515000739,19990824000003,19990626000007
1,005930,dcm_no,7861003,7549051,7446167,7341446,7202455,6958001,6846651,6738798,...,263890,237424,178927,206571,113424,89871,41243,60524,22086,15402
2,005380,rcp_no,20210316000986,20201116001588,20200814002203,20200515002179,20200330004041,20191114001732,20190814002230,20190515002011,...,20011113000302,20010816000050,20010330000582,20010514000734,20001114000003,20000814000121,20000503000073,20000515000428,19991209000001,19990407000005
3,005380,dcm_no,7880318,7549889,7447766,7343256,7203672,6959184,6846694,6739815,...,262569,237756,174571,205881,112238,89163,55439,60064,24529,10115
4,017670,rcp_no,20210317000804,20201116000869,20200814002234,20200515002457,20200330003443,20191114001848,20190814002530,20190515001803,...,20011114000904,20010814000621,20010331000096,20010515000510,20001114000690,20000814000992,20000330000523,20000515000257,19990813000219,19990610000004
5,017670,dcm_no,7884796,7548156,7447875,7344012,7199777,6959482,6847756,6739285,...,264246,236424,178107,206530,113330,90617,40681,59813,20448,13386


# 재무제표 다운로드

In [86]:
r=rcp_df['2006.12'][2]
d=rcp_df['2006.12'][3]

In [81]:
error_df = pd.DataFrame(index = ['stock_code','year','rcp','dcm'])

for y in range(int(len(rcp_df.index)/2)):
    rcp_list_temp = rcp_df.iloc[2*y,2:].values
    dcm_list_temp = rcp_df.iloc[2*y+1,2:].values
    year_list_temp = rcp_df.columns[2:]
    stock_code_temp = rcp_df.iloc[2*y,0]
    for year, rcp, dcm in zip(year_list_temp,rcp_list_temp,dcm_list_temp):
        try:
            download_ftable(stock_code_temp,year,rcp,dcm)
        except:
            no_error=len(error_df.columns)
            error_df[no_error] = [stock_code_temp,year,rcp,dcm]

In [87]:
download_ftable('005380','2010',r,d)

ValueError: File is not a recognized excel file

In [78]:
def download_ftable(stock_code,year,rcp,dcm):
    url_download = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko'.format(rcp,dcm)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    resp_download = requests.get(url_download, headers = {'user-agent' : user_agent}, verify=False)
    table_download = BytesIO(resp_download.content)
    sheet_names = pd.ExcelFile(table_download).sheet_names
    sheets_group = [['연결 재무상태표','재무상태표','대차대조표'],['연결 현금흐름표','현금흐름표','현금흐름표'],['연결 포괄손익계산서','포괄손익계산서','손익계산서']]
    for sheets in sheets_group:
        i=0
        while True:
            if sheets[i] not in sheet_names:
                i=i+1
                if i > len(sheets):
                    print("재무상태표를 찾을 수 없습니다")
                    break
            else: 
                data_download = pd.read_excel(table_download, sheet_name = sheets[i])
                try:
                    data_download.to_csv("Raw/Dart_Crawling/"+stock_code+"_"+year+"_"+sheets[1]+".csv", encoding='euc-kr')
                except UnicodeEncodeError:
                    # 최근것은 euc-kr로 encoding되어 있으나, 2010년이전에는 utf-16으로 encoding된것으로 보임.
                    # 다만, utf-16에서 to_csv할 때 sep=','이 적용되지 않아 다운로드된 파일의 글씨는 깨지지만 utf-8로 사용.
                    data_download.to_csv("Raw/Dart_Crawling/"+stock_code+"_"+year+"_"+sheets[1]+".csv", encoding='utf-8')
                break

# ETC

In [85]:
# i=4
for i in range(10):
    while True:
        if i != 6:
            if i == 8:
                print('break')
                break
            i=i+1
        else:
            print(i)
            break

6
6
6
6
6
6
6
break
break


KeyboardInterrupt: 